In [27]:
import pandas as pd
import re

In [2]:
# pulling from pickle file created in step 1

In [28]:
df = pd.read_pickle('/Users/onb/Documents/Analysis/DoubleVerify Analysis/Data/Safety Flag/ZEFR_Safety_Flag_Risk_Data.pkl')

In [29]:
df.head()

,CHANNEL_ID,VIDEO_ID,SUITABILITY_CATEGORY,TAXONOMY_NAME,SCORE,DECISION,DV_RISK,vector1,vector2,simscore
0,UChxJPnZ0x9I8iYrm4jjuo0w,c1AGFOjKDbM,Religion & Spirituality,4A - Terrorism - [High],0.000000,False,Minimal,"{'Religion': 1, 'Spirituality': 1}","{'4A': 1, 'Terrorism': 1, 'High': 1}",0.0
4,UCwADl1l8G7IQxasyu43yi1A,9MLp5gi59dM,Celebrity Gossip,Brand Safety - Crime - V1,0.043045,None,Minimal,"{'Celebrity': 1, 'Gossip': 1}","{'Brand': 1, 'Safety': 1, 'Crime': 1, 'V1': 1}",0.0
11,UC2c6FqF4olAZ7LodVEUueNQ,XHNZn7A4mcY,Family & Parenting: Kids Content,4A - Death & Injury - [Medium],0.000000,False,Minimal,"{'Family': 1, 'Parenting': 1, 'Kids': 1, 'Cont...","{'4A': 1, 'Death': 1, 'Injury': 1, 'Medium': 1}",0.0
13,UCvRAX-ujvZ0eTMLGG2vki9w,zpp9fkHTzis,"Law, Gov't & Politics: Politics",Japanese Brand Safety - V1,0.134686,True,Minimal,"{'Law': 1, 'Gov': 1, 't': 1, 'Politics': 2}","{'Japanese': 1, 'Brand': 1, 'Safety': 1, 'V1': 1}",0.0
15,UCJQXB7cxKeHGaMRig179adg,8yC0jpwUoeQ,Celebrity Gossip,4A - Social Issues - [High],0.000000,False,Minimal,"{'Celebrity': 1, 'Gossip': 1}","{'4A': 1, 'Social': 1, 'Issues': 1, 'High': 1}",0.0


In [30]:
df.drop(['vector1','vector2'], axis = 1, inplace = True)

In [31]:
concepts = []

for txt in df['TAXONOMY_NAME']:
    if '4A' in txt:
        result =  re.search('- (.*) -', txt)
        concepts.append(result.group(1))
    else:
        concepts.append(txt)
        
df['CONCEPTS'] = concepts

In [32]:
ZEFR_RISK = []

for txt in df['TAXONOMY_NAME']:
    if 'Minimal' in txt:
        ZEFR_RISK.append('Minimal')
    elif 'Low' in txt:
        ZEFR_RISK.append('Low')
    elif 'Medium' in txt:
        ZEFR_RISK.append('Medium')
    elif 'High' in txt:
        ZEFR_RISK.append('High')
    else:
        ZEFR_RISK.append('No GARM')
        
df['ZEFR_RISK'] = ZEFR_RISK

In [33]:
same_risk = []

for i,v in enumerate(df['TAXONOMY_NAME']):
    if 'Minimal' in df['DV_RISK'].iloc[i] and 'Minimal' in v:
        same_risk.append(1)
    elif 'Low' in df['DV_RISK'].iloc[i] and 'Low' in v:
        same_risk.append(1)  
    elif 'Medium' in df['DV_RISK'].iloc[i] and 'Medium' in v:
        same_risk.append(1) 
    elif 'High' in df['DV_RISK'].iloc[i] and 'High' in v or 'Floor' in v:
        same_risk.append(1) 
    else:
        same_risk.append(0)

In [34]:
df['SAME_RISK'] = same_risk

In [35]:
df = df[df['ZEFR_RISK']!='No GARM']

In [37]:
df.to_clipboard()

In [38]:
df.head()

,CHANNEL_ID,VIDEO_ID,SUITABILITY_CATEGORY,TAXONOMY_NAME,SCORE,DECISION,DV_RISK,simscore,CONCEPTS,ZEFR_RISK,SAME_RISK
0,UChxJPnZ0x9I8iYrm4jjuo0w,c1AGFOjKDbM,Religion & Spirituality,4A - Terrorism - [High],0.0,False,Minimal,0.0,Terrorism,High,0
11,UC2c6FqF4olAZ7LodVEUueNQ,XHNZn7A4mcY,Family & Parenting: Kids Content,4A - Death & Injury - [Medium],0.0,False,Minimal,0.0,Death & Injury,Medium,0
15,UCJQXB7cxKeHGaMRig179adg,8yC0jpwUoeQ,Celebrity Gossip,4A - Social Issues - [High],0.0,False,Minimal,0.0,Social Issues,High,0
16,UCJQXB7cxKeHGaMRig179adg,8yC0jpwUoeQ,Society: Marriage,4A - Social Issues - [High],0.0,False,Minimal,0.0,Social Issues,High,0
23,UCkScchDI48GkQxIhdkSUrgQ,1S3kURkfvnM,Family & Parenting: Kids Content,4A - Crime - [Minimal],1.0,True,Minimal,0.0,Crime,Minimal,1


In [1]:
df['SUITABILITY_CATEGORY'].unique()

NameError: name 'df' is not defined

In [39]:
import snowflake.connector
from snowflake.connector.pandas_tools import write_pandas
from snowflake.sqlalchemy import URL
from sqlalchemy import create_engine
from snowflake.connector.pandas_tools import pd_writer

In [40]:
table_name = 'ZEFR_SAFETY_FLAG_RISK'
schema = 'ANALYST'

In [41]:
df.rename(columns={"simscore": "SIMSCORE"}, inplace = True)

In [42]:
def connection(schema):
    conn = snowflake.connector.connect(
        host="localhost",
        port="10015",
        user="sdm",
        password="sdm",
        account="dummy",
        protocol="http",
        warehouse="ENGINEER_WH",
        database="PROD",
        schema=schema,
    )
    return conn

In [43]:
def write_snowflake(schema,df,table_name):
    conn = connection(schema)
    success, nchunks, nrows, _ = write_pandas(conn, df, table_name, chunk_size = 1000)
    
    return success, nchunks, nrows, _

In [44]:
success, nchunks, nrows, _ = write_snowflake(schema, df, table_name)

ProgrammingError: 000904 (42000): SQL compilation error: error line 1 at position 199
invalid identifier 'CONCEPTS'